## Introduction to functional API

Let's start with a minimal example that shows side by side a simple Sequential model and its
equivalent in the functional API

In [22]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input
import numpy as np

In [19]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64, )))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))

In [20]:
input_tensor = Input(shape = (64, ))
x = layers.Dense(32, activation = 'relu')(input_tensor)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [23]:
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

In [24]:
model.fit(x_train, y_train)

Epoch 1/1
1000/1000 [==============================] - 0s 167us/step - loss: 14.0316


In [25]:
score = model.evaluate(x_train, y_train)
score

1000/1000 [==============================] - 0s 65us/step


In [29]:
predictions = model.predict(x_train)
predictions[:2, ]

array([[0.46121794, 0.12121745, 0.01150203, 0.30818877, 0.01422315,
        0.00501559, 0.00118386, 0.02001156, 0.05508405, 0.00235549],
       [0.4334674 , 0.08721125, 0.0151925 , 0.35336727, 0.01655284,
        0.00547198, 0.0009094 , 0.0224867 , 0.06411348, 0.0012271 ]],
      dtype=float32)

## Multi-input models
Following is an example of how you can build such a model with the functional API. You set up two independent brances, encoding the tect input and the question input as representation vectors; then, concatenate these vectors; and finally, add a softmax classifier on top of the concatenated representations.

In [60]:
# Functional API implementation of a two-input question-answering model
from keras.models import Model
from keras import layers
from keras import Input

In [61]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

In [62]:
text_input = Input(shape = (None, ), dtype = 'int32', name = 'text')
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

In [63]:
question_input = Input(shape = (None, ), dtype = 'int32', name = 'question')
embedded_question = layers.Embedding(64, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

In [64]:
concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)

In [65]:
answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)

In [66]:
model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

How do you train this two-input model? There are two possible APIs: you can feed the model a list of Numpy arrays as inputs, or you can feed it a dictionary that maps unput names to Numpy arrays. Naturally, the latter option is available only if you give names to your inputs

In [67]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
question = np.random.randint(1, answer_vocabulary_size, size = (num_samples, max_length))
answers = np.random.randint(0, 2, size = (num_samples, answer_vocabulary_size))

In [68]:
model.fit([text, question], answers, epochs = 10, batch_size = 128)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1556.4727 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1561.7858 - acc: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1565.3442 - acc: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1567.3065 - acc: 0.0000e+00
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1568.8153 - acc: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1570.0150 - acc: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1571.0145 - acc: 0.0000e+00
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1571.8384 - acc: 0.0000e+00
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1572.5224 - acc: 0.0000e+00
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/

In [69]:
model.fit({'text': text, 'question' : question}, answers, epochs = 10, batch_size = 128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1573.5332 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1573.9036 - acc: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1574.2746 - acc: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1574.5398 - acc: 0.0000e+00
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1574.8030 - acc: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1574.9612 - acc: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1575.1699 - acc: 0.0000e+00
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1575.2938 - acc: 0.0000e+00
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 1575.4331 - acc: 0.0000e+00
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/